<a href="https://colab.research.google.com/github/chottokun/colaboratory/blob/main/duplicate_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#

In [2]:
!mkdir origin
!mkdir target

In [3]:
import filecmp
import sys
import shutil
from pathlib import Path

# 参考
# https://cfd.life/column/python%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC%E3%81%8C%E5%85%A8%E3%81%A6%E6%AD%A3%E5%B8%B8%E3%81%AB%E8%A1%8C%E3%81%88%E3%81%A6%E3%81%84%E3%82%8B%E4%BA%8B%E3%82%92/

class compare_result:
    def __init__(self,
                 sub_left_only,
                 sub_right_only,
                 sub_diff_files,
                 sub_funny_files,
                ):

        self.left_only=sub_left_only
        self.right_only=sub_right_only
        self.diff_files=sub_diff_files
        self.funny_files=sub_funny_files

def compare_dirs(dir1, dir2, ignore=None):
    dirs_equal = filecmp.dircmp(dir1, dir2, ignore=ignore)

    left_only = dirs_equal.left_only
    right_only = dirs_equal.right_only
    diff_files = dirs_equal.diff_files
    funny_files = dirs_equal.funny_files

    for subdir in dirs_equal.common_dirs:

        subdir1 = f"{dir1}/{subdir}"
        subdir2 = f"{dir2}/{subdir}"

        cd = compare_dirs(subdir1, subdir2, ignore)

        left_only += [f"{subdir}/{f}" for f in cd.left_only]
        right_only += [f"{subdir}/{f}" for f in cd.right_only]
        diff_files += [f"{subdir}/{f}" for f in cd.diff_files]
        funny_files += [f"{subdir}/{f}" for f in cd.funny_files]

    return compare_result(left_only, right_only, diff_files, funny_files)
    # yield compare_result(left_only, right_only, diff_files, funny_files)

def get_origin_files(cd):
    for cp in cd.right_only:
        parent_path = Path(cp)
        origin_path = origin_folder / cp
        target_path = target_folder / parent_path.parent

        if not origin_path.is_file():
            for _path in Path(origin_path).glob("**/*"):
                # print(_path)
                yield _path, target_path / cp
            continue

        yield origin_path, target_path


if __name__=="__main__":

    # 引数
    args = sys.argv

    #
    target_folder = Path("./target") if args[0] != None else args[0]
    origin_folder = Path("./origin") if len(args) << 2 else args[1]

    filecmp.clear_cache()

    cd=compare_dirs(target_folder, origin_folder, ignore=["$RECYCLE.BIN"])

    # print("Left only:")
    # print(cd.left_only)
    # print("Right only:")
    # print(cd.right_only)
    # print("Different files:")
    # print(cd.diff_files)
    # print("Funny files:")
    # print(cd.funny_files)

    for _op, _tp in get_origin_files(cd):
        print(_op, _tp)
        print(f'MB: {_op.stat().st_size / 1048576:.3f}')
        print((1 * (_op.stat().st_size / 1048576)), "sec")
        # shutil.copy2() でコピーする。
